In [6]:
# adapted ameea's code to clean the data retrieved from:
#https://www.kaggle.com/ameeamin/d/snap/amazon-fine-food-reviews/using-k-means-clustering-to-predict-helpfulness/comments
# Using 'Reviews.csv' file at https://www.kaggle.com/snap/amazon-fine-food-reviews 
#imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# read data into a DataFrame
data = pd.read_csv("Reviews.csv")

#make a copy of columns we need from raw data
dfFineFood = data.iloc[:, [1,2,3,4,5,6,7,8,9]]
dfFineFood.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
#change data type of non-Text features from string to integer
dfFineFood.iloc[:, 3:5] = dfFineFood.iloc[:, 3:5].apply(pd.to_numeric)

/Users/majed/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
dfFineFood['Time'] = pd.to_datetime(dfFineFood['Time'],unit='s')
dfFineFood.loc[:,'Score'].shape

/Users/majed/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(568454,)

In [9]:
# convert text to lowercase
#https://www.kaggle.com/ameeamin/d/snap/amazon-fine-food-reviews/using-k-means-clustering-to-predict-helpfulness/comments
dfFineFood.loc[:, 'Summary'] = dfFineFood.loc[:,'Summary'].str.lower()
dfFineFood.loc[:, 'Text'] = dfFineFood.loc[:,'Text'].str.lower()
dfFineFood.head(10)

/Users/majed/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,good quality dog food,i have bought several of the vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,not as advertised,product arrived labeled as jumbo salted peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""delight"" says it all",this is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,cough medicine,if you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,great taffy,great taffy at a great price. there was a wid...
5,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,2012-07-12,nice taffy,i got a wild hair for taffy and ordered this f...
6,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,2012-06-20,great! just as good as the expensive brands!,this saltwater taffy had great flavors and was...
7,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,2012-05-03,"wonderful, tasty taffy",this taffy is so good. it is very soft and ch...
8,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,2011-11-23,yay barley,right now i'm mostly just sprouting this so my...
9,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,2012-10-26,healthy dog food,this is a very healthy dog food. good for thei...


In [ ]:
#https://www.kaggle.com/ameeamin/d/snap/amazon-fine-food-reviews/using-k-means-clustering-to-predict-helpfulness/comments
#remove html tags
# https://pypi.python.org/pypi/bleach
# I have an issue with this code because it takes for ever to process
#import bleach
#df1["Text"] = df1['Text'].apply(lambda x: bleach.clean(x, tags=[], strip=True))
#df1["Text"].head(4)

In [10]:
#https://www.kaggle.com/ameeamin/d/snap/amazon-fine-food-reviews/using-k-means-clustering-to-predict-helpfulness/comments
#remove punctuation
import unicodedata
import sys

tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))
def remove_punctuation(text):
    return text.translate(tbl)

dfFineFood.loc[:,'Text']=dfFineFood.loc[:,'Text'].apply( lambda x: remove_punctuation(x))
dfFineFood.loc[:,"Text"].head(4)

/Users/majed/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
Name: Text, dtype: object

In [11]:
# exported to a dataset
dfFineFood.to_csv('dfFineFood_Output.csv')

In [18]:
# read another Amazon Dataset from : 
#http://jmcauley.ucsd.edu/data/amazon/
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

dfElectronics = getDF('reviews_Electronics_5.json.gz')
dfElectronics.head(2)

,reviewText,overall,unixReviewTime,reviewerID,reviewerName,helpful,summary,reviewTime,asin
0,We got this GPS for my husband who is an (OTR)...,5.0,1370131200,AO94DHGC771SJ,amazdnu,"[0, 0]",Gotta have GPS!,"06 2, 2013",0528881469
1,"I'm a professional OTR truck driver, and I bou...",1.0,1290643200,AMO214LNFCEI4,Amazon Customer,"[12, 15]",Very Disappointed,"11 25, 2010",0528881469


In [19]:
dfElectronics = dfElectronics.iloc[:, [0,1,2,3,4,5,6]]
dfElectronics.head()

,reviewText,overall,unixReviewTime,reviewerID,reviewerName,helpful,summary
0,We got this GPS for my husband who is an (OTR)...,5.0,1370131200,AO94DHGC771SJ,amazdnu,"[0, 0]",Gotta have GPS!
1,"I'm a professional OTR truck driver, and I bou...",1.0,1290643200,AMO214LNFCEI4,Amazon Customer,"[12, 15]",Very Disappointed
2,"Well, what can I say. I've had this unit in m...",3.0,1283990400,A3N7T0DY83Y4IG,C. A. Freeman,"[43, 45]",1st impression
3,"Not going to write a long review, even thought...",2.0,1290556800,A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","[9, 10]","Great grafics, POOR GPS"
4,I've had mine for a year and here's what we go...,1.0,1317254400,A24EV6RXELQZ63,Wayne Smith,"[0, 0]","Major issues, only excuses for support"


In [20]:
dfElectronics.iloc[:, 1] = dfElectronics.iloc[:, 1].apply(pd.to_numeric)

In [21]:
dfElectronics.loc[:, 'reviewText'] = dfElectronics.loc[:,'reviewText'].str.lower()
dfElectronics.loc[:,"reviewText"].head(5)

0    we got this gps for my husband who is an (otr)...
1    i'm a professional otr truck driver, and i bou...
2    well, what can i say.  i've had this unit in m...
3    not going to write a long review, even thought...
4    i've had mine for a year and here's what we go...
Name: reviewText, dtype: object

In [22]:
dfElectronics.to_csv('dfElectronics_Output.csv')

In [ ]:
dfFineFood_df = sqlContext.read\
    .format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .option("nullValue", 'NA')\
    .load("file:///home/ischool/dfFineFood_Output.csv")